In [1]:
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from tqdm import tqdm
import torch.optim as optim
import yaml
import pickle
from torch.utils.data import Dataset, DataLoader

Carico il configuratore che tiene traccia di tutte le informazioni generali 

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None
with open("./config.yaml", 'r') as f:
    config = yaml.safe_load(f)

# Download Dati

In questa parte carico i `.csv` e gli organizzo in modo tale che siano ordinati per

`(data, denominazione_regionale, denominazione_provincia)`

Inoltre pulisco i vari datase poiché ci sono una serie di righe che presentano delle problematicità in termini di valori.

La matrice di adiacenza generale è stata creata connettendo tutte le provincie di una determinata regione, in modo tale da mantenere una similitudine con il problema originario. Inoltre, per non distaccarsi troppo con la realtà, sono state inserite delle connessioni tra regioni dato che quest'ultime non sono isolate. In particolare sono state aggiunte connessioni tra tutti i capoluoghi.

In [3]:
def get_adj(config: yaml):
    if os.path.exists(os.path.join(config['paths']['adj'],"adj_totale.pkl")):
        with open(os.path.join(config['paths']['adj'],"adj_totale.pkl"), "rb") as f:
            adj = pickle.load(f)
    else:
        codice_reg_prov = pd.read_csv(os.path.join(config['paths']['data'], "codice_reg_prov.csv"), index_col=0)

        # Carico un dizionario in cui le chiavi sono le regioni e i valori sono i rispettivi capoluohi
        with open(os.path.join(config['paths']['data'], "capoluoghi.json"), 'r') as f:
            capoluoghi = json.load(f)
        
        adjs = {}
        adj = np.zeros((len(codice_reg_prov), len(codice_reg_prov)))
        i = 0
        index_capoluoghi = []
        d = codice_reg_prov.groupby('denominazione_provincia').codice_provincia.max().to_dict()
        regioni = codice_reg_prov.groupby(by=['codice_regione']).codice_provincia.unique()
        
        for cod_reg in tqdm(regioni.index):
            codici_provincia = regioni[cod_reg]
            n_prov = len(codici_provincia)
            adjs[cod_reg] = np.ones((n_prov, n_prov))-np.eye(n_prov)
            adj[i:i+n_prov, i:i+n_prov] = adjs[cod_reg]

            region = codice_reg_prov[codice_reg_prov.codice_regione==cod_reg].denominazione_regione.values[0]
            codice_capoluogo = codice_reg_prov[codice_reg_prov.denominazione_provincia==capoluoghi[region]].codice_provincia.values[0]
            pos = np.where(codici_provincia == codice_capoluogo)[0][0]
            index_capoluoghi.append(i+pos)
            
            
            with open(os.path.join(config['paths']['adj'],f"adj_{region}.pkl"), "wb") as f:
                pickle.dump(adj, f)
                
            i += n_prov
        for ind_r in index_capoluoghi:
            for ind_c in index_capoluoghi:
                if ind_r != ind_c:
                    adj[ind_r, ind_c] = 1
                
        with open(os.path.join(config['paths']['adj'],"adj_totale.pkl"), "wb") as f:
            pickle.dump(adj, f)
        
    return adj


In [4]:
def fill_dataset(df: pd, 
                 config:yaml):
    # questa procedura serve solo ad inserire i dati sulle province mancanti
    # questa procedura serve solo ad inserire i dati sulle province mancanti
    d = []
    feat = df.shape[1]
    codici = list(df.codice_provincia.unique())
    codice_reg_prov = {tuple(x) for x in df[config['dataset']['col_categorical_prov']].values.tolist()}
    codice_reg_prov = pd.DataFrame(codice_reg_prov, columns=config['dataset']['col_categorical_prov'])
    codice_reg_prov = codice_reg_prov.sort_values(by = ['codice_regione', 'codice_provincia']).reset_index(drop = True)
    
    codici = list(codice_reg_prov.codice_provincia.unique())
    for data in tqdm(df.data.unique(), desc = "filling"):
        tmp = df[df.data == data]
        if len(tmp) != len(codici):
            # individue le province mancanti
            codici_mancanti = [x for x in codici if x not in tmp.codice_provincia.unique()]
            tmp = pd.concat([tmp, codice_reg_prov[codice_reg_prov.codice_provincia.isin(codici_mancanti)]])        
            tmp.data = data
            tmp = tmp.fillna(0)
    
            if len(tmp) != len(codici):
                import pdb
                pdb.set_trace()
        d.append(tmp)
    df = pd.concat(d).sort_values(by = ['data','codice_regione', 'codice_provincia']).reset_index(drop = True)
    df = df[list(df.columns.drop(['codice_regione', 'codice_provincia']))+['codice_regione', 'codice_provincia']]
    return df

def get_dataset_regioni(config:yaml):
    if os.path.exists(os.path.join(config['paths']['data'], "covid_regioni.csv")):
            regions = pd.read_csv(os.path.join(config['paths']['data'],"covid_regioni.csv"), index_col=0)
            regions['data'] = pd.to_datetime(regions['data'], format='%Y-%m-%d')
    else:
        ########## I create take the dataset for the regions
        data = []
        for csv in tqdm(os.listdir(os.path.join(config['paths']['raw_data'],"dati-regioni")), desc = "regione"):
            if csv.split(".")[-1] == "csv":
                data.append(pd.read_csv(os.path.join(config['paths']['raw_data'],"dati-regioni",csv)))
        data = pd.concat(data).drop_duplicates().reset_index(drop = True)
        
        # Trento
        data.codice_regione = data.codice_regione.replace(21, 4)
        # Bolzano
        data.codice_regione = data.codice_regione.replace(22, 4)
        # in questo modo sommo tutte le variabili delle due provincie
        # non ci sono problemi perché le variabili rimanenti sono solo numeriche 
        data = data.groupby(by = ['data', 'codice_regione']).sum().reset_index()
        
        regions = data[['data', 'codice_regione', 'ricoverati_con_sintomi', 'terapia_intensiva', 'totale_ospedalizzati',
           'isolamento_domiciliare', 'dimessi_guariti', 'deceduti']]
        regions.data = regions.data.apply(lambda x: x.split("T")[0])
        regions.data = pd.to_datetime(regions['data'], format='%Y-%m-%d')
        
        regions = regions.drop_duplicates().reset_index(drop = True)
        regions.to_csv(os.path.join(config['paths']['data'],"covid_regioni.csv"))
    return regions
def get_dataset_province(config:yaml):
    if os.path.exists(os.path.join(config['paths']['data'],"covid_province.csv")):
            provincials = pd.read_csv(os.path.join(config['paths']['data'],"covid_province.csv"), index_col = 0)
            provincials['data'] = pd.to_datetime(provincials['data'], format='%Y-%m-%d')
    else:
        data = []
        for csv in tqdm(os.listdir(os.path.join(config['paths']['raw_data'],"dati-province")), desc = "provincia"):
            if csv.split(".")[-1] == "csv":
                tmp = pd.read_csv(os.path.join(config['paths']['raw_data'],"dati-province",csv), 
                                  usecols = ['data', 'codice_regione', 'codice_provincia', 'totale_casi'])
                data.append(tmp)
        
        data = pd.concat(data)
        data.reset_index(drop = True, inplace=True)
        data.data = data.data.apply(lambda x: x.split("T")[0])
        data.data = pd.to_datetime(data['data'], format='%Y-%m-%d')
        
        # Riduco il dataset con le variabili che servono
        # Inoltre trasformo la conta totale dei positivi per ogni regione in nuovi positivi
        #data.rename(columns={'totale_casi': 'nuovi_casi'}, inplace=True)
        provincials = data.drop_duplicates()
        provincials = provincials[-(provincials.codice_provincia>200)]
        
        # codici relative alle province che sono state male elaborate
        provincials = provincials[-provincials.codice_regione.isin([21,22])]
        
        ############## creo la variazione giornaliera dei contagi  ##########################à
        provincials = provincials[['data','codice_regione','codice_provincia', 'totale_casi']].sort_values(by = ['data', 'codice_provincia'])
        provincials['nuovi_casi']= provincials.groupby('codice_provincia').diff().totale_casi
        provincials = provincials.dropna().reset_index(drop = True)
        # Siccome ci sono giorni in cui il numero di positivi è negativo per via di errori
        # segno quei valori a zero e quindi il totale positivi rimane invariato 
        print("hello")
        provincials[provincials.nuovi_casi<0].totale_casi = provincials.loc[list(provincials[provincials.nuovi_casi<0].index-1)].totale_casi
        provincials.nuovi_casi = provincials.nuovi_casi.apply(lambda x: max(0, x))
        provincials.to_csv(os.path.join(config['paths']['data'],"covid_province.csv"))
    
    return provincials
    
def get_dataset_covid(config: yaml):
    # Creo il dataset completo 
    if os.path.exists(os.path.join(config['paths']['data'],"covid.csv")):
        df = pd.read_csv(os.path.join(config['paths']['data'], "covid.csv"), index_col=0)
        df['data'] = pd.to_datetime(df['data'], format='%Y-%m-%d')

    else:
        provincials = get_dataset_province(config)
        regions = get_dataset_regioni(config)
        df = pd.merge(provincials, regions, 
                      how = "inner",  
                      on = ['data','codice_regione']) 
        df = df.sort_values(by = ['data','codice_regione', 'codice_provincia']).reset_index(drop = True)
        codice_reg_prov = {tuple(x) for x in df[config['dataset']['col_categorical_prov']].values.tolist()}
        codice_reg_prov = pd.DataFrame(codice_reg_prov, columns=config['dataset']['col_categorical_prov'])

        ################# Filling the dataset ###################
        df = fill_dataset(df, config)

        ################# Date variables ########################
        df['year'] = df.data.dt.year-df.data.dt.year.min()
        df['month'] = df.data.dt.month-df.data.dt.month.min()
        df['day'] = df.data.dt.day-df.data.dt.day.min()
        df['day_of_week'] = df.data.dt.dayofweek
        df['festivo'] = 0
        df[df.day_of_week>5].festivo = 1

        ################# variation ##########
        num_variables = df.columns[3:-7]
        tmp=df.groupby('codice_provincia').diff()[num_variables]
        tmp.columns = [f"variation_{x}" for x in tmp.columns]
        df = pd.concat([df.data, tmp, df.drop(columns = 'data')], 1)
        df = df.dropna().reset_index(drop=True)
        codice_reg_prov.to_csv(os.path.join(config['paths']['data'],"codice_reg_prov.csv"))
        df.to_csv(os.path.join(config['paths']['data'],"covid.csv"))
    return df
    
data = get_dataset_covid(config)
data.head()

,data,variation_ricoverati_con_sintomi,variation_terapia_intensiva,variation_totale_ospedalizzati,variation_isolamento_domiciliare,variation_dimessi_guariti,variation_deceduti,totale_casi,nuovi_casi,ricoverati_con_sintomi,...,isolamento_domiciliare,dimessi_guariti,deceduti,codice_regione,codice_provincia,year,month,day,day_of_week,festivo
0,2020-02-26,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,2.0,...,1.0,0.0,0.0,1,1,0,1,25,2,0
1,2020-02-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,1.0,0.0,0.0,1,2,0,1,25,2,0
2,2020-02-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,1.0,0.0,0.0,1,3,0,1,25,2,0
3,2020-02-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,1.0,0.0,0.0,1,4,0,1,25,2,0
4,2020-02-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,1.0,0.0,0.0,1,5,0,1,25,2,0
